# task 2- Archiving Voters Details from Local Election 2079

–  Archive the Voters Details from the Local Election 2079, Nepal
Refer the website (https://localelection2079.ekantipur.com/?lng=eng) and scrap the voters
details such as country( i.e. Nepal), election_name (i.e. Local Election 2079), province, district,
municipality (i.e. Kathmandu Metropolition), Total Population, Number of Wards, Election
Center, Number of Male Voters, Number of Female Voters, Number of Other Voters, Total
Eligible Voters

### Data scraping using BeautifulSoup, Selenium and Python

In [88]:
# importing necessary libraries
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException, ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
import pandas as pd
from bs4 import BeautifulSoup


# Initialize WebDriver 
driver = webdriver.Chrome(service=Service("D:/pythonprogramme/selenium/chromedriver.exe"))

# Loadding page
driver.get('https://localelection2079.ekantipur.com/?lng=eng') 

# function for ward block
def select_wards_with_retry(retries=10):
    attempt = 0
    while attempt < retries:
        if driver.find_element(By.CSS_SELECTOR, '#c > a').text == 'Local Election 2079':
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="select-box"]/option'))
                )
                 # data scraping
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, "html.parser")

                if not soup.find_all('li', class_='breadcrumb-item')[3].text.strip()=="Dhorpatan Hunting Reserve Municipality":
                    country_ = "Nepal"
                    election_name_ = "Local Election 2079"
                    province_ = soup.find_all('li', class_='breadcrumb-item')[1].text.strip()
                    district_ = soup.find_all('li', class_='breadcrumb-item')[2].text.strip()
                    municipality_ = soup.find_all('li', class_='breadcrumb-item')[3].text.strip()
                    
                    tab_container = soup.find('div', class_="tab-container")
                    total_wards = len(driver.find_elements(By.XPATH, '//*[@id="select-box"]/option'))

                    wards_count = 0
                    for j, wada in enumerate(tab_container.find_all('div', class_='tab-content')):
                        _ward = j + 1

                        if j < total_wards:
                            wards_count += 1
                        else:
                            break
                    
                    basic_information.append([country_,election_name_,province_,district_,municipality_,wards_count])
                    print([country_,election_name_,province_,district_,municipality_,wards_count])

                    for data in soup.find_all('li',class_='list-group-item d-flex justify-content-between align-items-start'):
                        for i in data.find('span',class_='badge'):
                            indept_information.append(i.text.strip())
                            

                    
                    
                else:
                    break

                break

            except TimeoutException:
                attempt += 1
                while True:
                    try:
                        search_button = driver.find_element(By.XPATH, '//*[@id="search-result"]')
                        driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
                        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-result"]')))
                        driver.execute_script("arguments[0].click();", search_button)

                        WebDriverWait(driver, 20).until(
                            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "body > div.container > div.card.p-3.my-4 > div > div.col-xs-12.col-lg > div.card-body.section-1"))
                        )

                        if driver.find_element(By.CSS_SELECTOR, '#c > a').text == 'Local Election 2079':
                            # Get page source and parse
                            page_source = driver.page_source
                            soup = BeautifulSoup(page_source, "html.parser")

                            if not soup.find_all('li', class_='breadcrumb-item')[3].text.strip()=="Dhorpatan Hunting Reserve Municipality":
                                country_ = "Nepal"
                                election_name_ = "Local Election 2079"
                                province_ = soup.find_all('li', class_='breadcrumb-item')[1].text.strip()
                                district_ = soup.find_all('li', class_='breadcrumb-item')[2].text.strip()
                                municipality_ = soup.find_all('li', class_='breadcrumb-item')[3].text.strip()
                                _ward_=""
                            
                                basic_information.append([country_,election_name_,province_,district_,municipality_,_ward_])
                                print([country_,election_name_,province_,district_,municipality_,_ward_])

                                for data in soup.find_all('li',class_='list-group-item d-flex justify-content-between align-items-start'):
                                    for i in data.find('span',class_='badge'):
                                        indept_information.append(i.text.strip())
                                        

                            else:
                                break

                        else:
                            driver.execute_script("window.history.go(-1)")

                        break
                        
                    except (ElementClickInterceptedException, StaleElementReferenceException, TimeoutException):
                        continue

                if attempt >= retries:
                    print("Failed to load ward options after multiple retries.")
                    break

        else:
            driver.execute_script("window.history.go(-1)")


# function for skipped pages
def retry_skipped_pages(skipped_pages, max_retries=5):
    for page in skipped_pages:
        attempt = 0
        success = False
        while attempt < max_retries and not success:
            try:
                driver.get(page)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.container > div.card.p-3.my-4 > div > div.col-xs-12.col-lg > div.card-body.section-1"))
                )
                
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, "html.parser")

                country_ = "Nepal"
                election_name_ = "Local Election 2079"
                province_ = soup.find_all('li', class_='breadcrumb-item')[1].text.strip()
                district_ = soup.find_all('li', class_='breadcrumb-item')[2].text.strip()
                municipality_ = soup.find_all('li', class_='breadcrumb-item')[3].text.strip()
                _ward_=""

                basic_information.append([country_,election_name_,province_,district_,municipality_,_ward_])
                print("This is from skipped pages at last")
                print(country_,election_name_,province_,district_,municipality_,_ward_)

                for data in soup.find_all('li',class_='list-group-item d-flex justify-content-between align-items-start'):
                    for i in data.find('span',class_='badge'):
                        indept_information.append(i.text.strip())
                        
                        
                
                select_wards_with_retry()
                
                success = True

            except (TimeoutException, StaleElementReferenceException, NoSuchElementException):
                attempt += 1
                print(f"Retrying {page} ({attempt}/{max_retries})")

            if attempt >= max_retries:
                print(f"Failed to retrieve {page} after {max_retries} attempts.")



skipped_pages = []
pro_index = 0
dist_index = 0
muni_index = 0
basic_information=[]
indept_information=[]


WebDriverWait(driver, 120).until(
    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="provinceDD"]/option'))
)

# Province Selection
province_options = driver.find_elements(By.XPATH, "//*[@id='provinceDD']/option")[1:]
while pro_index < len(province_options):
    try:
        province_options = driver.find_elements(By.XPATH, "//*[@id='provinceDD']/option")[1:]
#         print("province:",province_options[pro_index].text)
        province_options[pro_index].click()
        time.sleep(1)
        
        # District Selection
        district_options = driver.find_elements(By.XPATH, '//*[@id="districtDD"]/option')[1:]
        while dist_index < len(district_options):
            district_options = driver.find_elements(By.XPATH, '//*[@id="districtDD"]/option')[1:]
            district_options[dist_index].click()
#             print("distirct:",district_options[dist_index].text)
            time.sleep(1)
            
            # Municipality Selection
            municipality_options = driver.find_elements(By.XPATH, '//*[@id="locationDD"]/option')[1:]
            while muni_index < len(municipality_options):
                municipality_options = driver.find_elements(By.XPATH, '//*[@id="locationDD"]/option')[1:]
#                 print("municipality:",municipality_options[muni_index].text)
                municipality_options[muni_index].click()
                time.sleep(1)
                
                # Click the search button
                while True:
                    try:
                        search_button = driver.find_element(By.XPATH, '//*[@id="search-result"]')
                        driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
                        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-result"]')))
                        driver.execute_script("arguments[0].click();", search_button)
                        break
                    except (ElementClickInterceptedException, StaleElementReferenceException):
                        continue

                select_wards_with_retry()

                muni_index += 1
            
            dist_index += 1
            muni_index = 0
            
        pro_index += 1
        dist_index = 0
        muni_index = 0
                
    except (NoSuchElementException, AttributeError,TimeoutException) as e:
        skipped_pages.append(driver.current_url)
        print("This page has some issues and will be retried at last")
        print(driver.current_url)
        
        if muni_index < len(municipality_options):
            muni_index += 1
        elif dist_index < len(district_options):
            dist_index += 1
            muni_index = 0
        else:
            pro_index += 1
            dist_index = 0
            muni_index = 0
            
        driver.execute_script("window.history.go(-1)")
        
retry_skipped_pages(skipped_pages)

driver.quit()

['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Aathrai Tribeni Rural Municipality', '']
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Aathrai Tribeni Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Pathibhara Yangwarak Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Phaktanglung Rural Municipality', '']
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Phaktanglung Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Phungling Municipality', 11]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Mikwakhola Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Meringden Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Maiwakhola Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Taplejung', 'Sidingba Rural Municipality', 7]
['Nepal', 'Local Election 2079', 

['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Chainpur Municipality', 11]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Dharmadevi Municipality', '']
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Dharmadevi Municipality', 9]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Panchakhapan Municipality', '']
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Panchakhapan Municipality', 9]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Bhotkhola Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Makalu Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Madi Municipality', 9]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Sabhapokhari Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Sankhuwasabha', 'Silichong Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Pradesh 1', 'Bhoj

['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Aurahi Municipality', '']
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Aurahi Municipality', 5]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Karjanha Municipality', 11]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Kalyanpur Municipality', 12]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Golbazar Municipality', 13]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Dhangadhimai Municipality', 14]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Naraha Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Nawarajpur Rural Municipality', '']
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Nawarajpur Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', 'Bariyarpatti Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Siraha', '

['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Paroha Municipality', 9]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Phatuwa Bijayapur Municipality', 11]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Baudhimai Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Madhav Narayan Municipality', 9]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Maulapur Municipality', 9]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Yemunamai Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Rajdevi Municipality', '']
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Rajdevi Municipality', '']
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Rajdevi Municipality', 9]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Rautahat', 'Rajpur Municipality', 9]
['Nepal', 'Local Election 2079', 'Madhesh Pradesh', 'Ra

['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Sindhupalchowk', 'Melamchi Municipality', 13]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Sindhupalchowk', 'Lisangkhu Pakhar Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Sindhupalchowk', 'Sunkoshi Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Sindhupalchowk', 'Helambu Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Kavrepalanchowk', 'Khanikhola Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Kavrepalanchowk', 'Chaurideurali Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Kavrepalanchowk', 'Temal Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Kavrepalanchowk', 'Dhulikhel Municipality', '']
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Kavrepalanchowk', 'Dhulikhel Municipality', 12]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Kavrepal

['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Makwanpur', 'Bhimphedhi Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Makwanpur', 'Makawanpurgadhi Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Makwanpur', 'Manahari Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Makwanpur', 'Raksirang Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Makwanpur', 'Hetauda Submetropolitan', '']
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Makwanpur', 'Hetauda Submetropolitan', 19]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Chitwan', 'Ichchhyakamana Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Chitwan', 'Kalika Municipality', 11]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Chitwan', 'Khairahani Municipality', 13]
['Nepal', 'Local Election 2079', 'Bagmati Pradesh', 'Chitwan', 'Bharatpur Metropolitan', 29]
['Nepal', 'Local Elect

['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Kanthekhola Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Galkota Municipality', 11]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Jaimini Municipality', 10]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Dhorpatan Municipality', 9]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Tamankhola Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Tarakhola Rural Municipality', '']
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Tarakhola Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Nisikhola Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Baglung Municipality', 14]
['Nepal', 'Local Election 2079', 'Gandaki Pradesh', 'Baglung', 'Badigad Rural Municipality', 10]
['Nepal', 'Local Election 2079', 'Gandaki Pr

['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Kapilvastu', 'Sudhodhan Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Ayirabati Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Gaumukhi Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Jhimruk Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Naubahini Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Pyuthan Municipality', 10]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Mallarani Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Mandavi Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Sarumarani Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Lumbini Pradesh', 'Pyuthan', 'Sworgadwary Municipality', 9]
['Nepal', 'Local Election 2079',

['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Surkhet', 'Lekbeshi Municipality', 10]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Surkhet', 'Birendranagar Municipality', 16]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Surkhet', 'Simta Rural Municipality', 9]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Aathabis Municipality', 9]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Gurans Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Chamunda Brindrasaini Municipality', 9]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Thantikandh Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Dungeshwor Rural Municipality', 6]
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Dullu Municipality', '']
['Nepal', 'Local Election 2079', 'Karnali Pradesh', 'Dailekh', 'Dullu Municipality', 13]
['Nepal', 'Local Election 2079', 'Karnali Pradesh'

['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Bajhang', 'Masta Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Bajhang', 'Saipal Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Bajhang', 'Surma Rural Municipality', 5]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Kamalbazar Municipality', 10]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Chaurpati Rural Municipality', 7]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Dhakari Rural Municipality', '']
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Dhakari Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Turmakhad Rural Municipality', 8]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Panchadewal Binayak Municipality', 9]
['Nepal', 'Local Election 2079', 'Sudurpaschim Pradesh', 'Achham', 'Bannigadhi Jayagadh Rural

## Creating dataframes

Webpage don't give data in formatted way, municipality's data is given is series so we scape data in such a way that to make two dataframe; df1 and df2.
Making that serial data into dataframe into df2 and concat them

In [89]:
df1=pd.DataFrame(basic_information,columns=['country','election name','province','district','municipality','number of wards'])

In [90]:
df1

,country,election name,province,district,municipality,number of wards
0,Nepal,Local Election 2079,Pradesh 1,Taplejung,Aathrai Tribeni Rural Municipality,
1,Nepal,Local Election 2079,Pradesh 1,Taplejung,Aathrai Tribeni Rural Municipality,5
2,Nepal,Local Election 2079,Pradesh 1,Taplejung,Pathibhara Yangwarak Rural Municipality,6
3,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phaktanglung Rural Municipality,
4,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phaktanglung Rural Municipality,7
...,...,...,...,...,...,...
848,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Marma Rural Municipality,6
849,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Malikarjun Municipality,8
850,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Lekam Rural Municipality,6
851,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Vyans Rural Municipality,6


### Creating dataframe from municipalities serial scraped data 

In [91]:
columns=['total population','wards','election center','number of male voters','number of female voters','number of other voters','total eligible voters']
df2 = pd.DataFrame([indept_information[i:i+len(columns)] for i in range(0, len(indept_information), len(columns))], columns=columns)

We can se that some municipality wards is not properly scraped because data is not present in website directly,
That's why we use a extra for loop to count total wards in a municipality and store it into wards_count 

In [ ]:
df2

### concating the two dataframe and make one combined dataframe

In [115]:
df=pd.concat([df1,df2],axis=1)

In [116]:
df

,country,election name,province,district,municipality,number of wards,total populaltion,wards,election center,number of male voters,number of female voters,drop column,total eligible voters
0,Nepal,Local Election 2079,Pradesh 1,Taplejung,Aathrai Tribeni Rural Municipality,,"12,288",5,5,"5,148","4,595",-,"9,743"
1,Nepal,Local Election 2079,Pradesh 1,Taplejung,Aathrai Tribeni Rural Municipality,5,"12,288",5,5,"5,148","4,595",-,"9,743"
2,Nepal,Local Election 2079,Pradesh 1,Taplejung,Pathibhara Yangwarak Rural Municipality,6,"11,797",6,6,"5,330","4,611",-,"9,941"
3,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phaktanglung Rural Municipality,,"11,925",7,12,"4,883","4,007",-,"8,890"
4,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phaktanglung Rural Municipality,7,"11,925",7,12,"4,883","4,007",-,"8,890"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Marma Rural Municipality,6,"15,586",6,14,"4,777","4,508",-,"9,285"
849,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Malikarjun Municipality,8,"15,754",-,15,"5,818","5,502",-,"11,320"
850,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Lekam Rural Municipality,6,"13,638",6,13,"5,016","4,754",-,"9,770"
851,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Vyans Rural Municipality,6,"10,205",-,11,"4,004","3,800",-,"7,804"


We need 753 rows or municipality but there is duplication of rows containing number of wards=" "
And after filtering those rows we get our derised dataframe

In [121]:
df = df[df['number of wards'] != ""]

In [11]:
df

,Unnamed: 0,country,election name,province,district,municipality,number of wards,total population,election center,number of male voters,number of female voters,number of other voters,total eligible voters
0,1,Nepal,Local Election 2079,Pradesh 1,Taplejung,Aathrai Tribeni Rural Municipality,5,12288,5,5148,4595,0,9743
1,2,Nepal,Local Election 2079,Pradesh 1,Taplejung,Pathibhara Yangwarak Rural Municipality,6,11797,6,5330,4611,0,9941
2,4,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phaktanglung Rural Municipality,7,11925,12,4883,4007,0,8890
3,5,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phungling Municipality,11,28786,11,8641,8561,0,17202
4,6,Nepal,Local Election 2079,Pradesh 1,Taplejung,Mikwakhola Rural Municipality,5,7991,6,3777,3190,0,6967
...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,848,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Marma Rural Municipality,6,15586,14,4777,4508,0,9285
748,849,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Malikarjun Municipality,8,15754,15,5818,5502,0,11320
749,850,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Lekam Rural Municipality,6,13638,13,5016,4754,0,9770
750,851,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Vyans Rural Municipality,6,10205,11,4004,3800,0,7804


### If duplicates rows are present then we remove it

In [130]:
df.drop_duplicates(inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_10908\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


We can see that one rows is decrease as removal of duplicate rows

In [131]:
df.shape

(752, 13)

Total no. of municipality = 753
But here is a problem that 1 municipality is missing, Benighat Rorang Rural Municipaity,Dhading district, province Bagmati does not contain any data in the website.
So total no. of municipality in my data have lagged 1,as containing 752 municipalities. 

In [133]:
df['municipality'].count()

752

Saving data into csv file as backup

In [14]:
df.to_csv("voters details from local election 2079.csv")

### droping unnecessary columns

In [12]:
df.drop(columns=[wards],inplace=True)

In [13]:
df

,country,election name,province,district,municipality,number of wards,total population,election center,number of male voters,number of female voters,number of other voters,total eligible voters
0,Nepal,Local Election 2079,Pradesh 1,Taplejung,Aathrai Tribeni Rural Municipality,5,12288,5,5148,4595,0,9743
1,Nepal,Local Election 2079,Pradesh 1,Taplejung,Pathibhara Yangwarak Rural Municipality,6,11797,6,5330,4611,0,9941
2,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phaktanglung Rural Municipality,7,11925,12,4883,4007,0,8890
3,Nepal,Local Election 2079,Pradesh 1,Taplejung,Phungling Municipality,11,28786,11,8641,8561,0,17202
4,Nepal,Local Election 2079,Pradesh 1,Taplejung,Mikwakhola Rural Municipality,5,7991,6,3777,3190,0,6967
...,...,...,...,...,...,...,...,...,...,...,...,...
747,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Marma Rural Municipality,6,15586,14,4777,4508,0,9285
748,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Malikarjun Municipality,8,15754,15,5818,5502,0,11320
749,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Lekam Rural Municipality,6,13638,13,5016,4754,0,9770
750,Nepal,Local Election 2079,Sudurpaschim Pradesh,Darchula,Vyans Rural Municipality,6,10205,11,4004,3800,0,7804


Adjusting the data types of some columns

In [8]:
# Convert  columns datatype to integers
columns_to_convert = [
    'number of wards',
    'total population',
    'election center',
    'number of male voters',
    'number of female voters',
    'number of other voters',
    'total eligible voters'
]

for col in columns_to_convert:
    # Replace non-numeric values with NaN
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
    # Replace NaN with 0
    df[col] = df[col].fillna(0).astype(int)


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   country                  752 non-null    object
 1   election name            752 non-null    object
 2   province                 752 non-null    object
 3   district                 752 non-null    object
 4   municipality             752 non-null    object
 5   total population         752 non-null    int32 
 6   number of wards          752 non-null    int32 
 7   election center          752 non-null    int32 
 8   number of male voters    752 non-null    int32 
 9   number of female voters  752 non-null    int32 
 10  number of other voters   752 non-null    int32 
 11  total eligible voters    752 non-null    int32 
dtypes: int32(7), object(5)
memory usage: 50.1+ KB


Just re-ordering the sequence of columns

In [19]:
columns_reordered = [
    'country', 
    'election name', 
    'province', 
    'district', 
    'municipality', 
    'total population', 
    'number of wards', 
    'election center', 
    'number of male voters', 
    'number of female voters',
    'number of other voters',
    'total eligible voters'
]

df = df[columns_reordered]

# Storing dataframe into database

### Importing some necessary libraries

In [15]:
import pandas as pd
import MySQLdb

### Connecting to database 

In [16]:
conn=MySQLdb.connect(
    host='localhost',
    user='root',
    password='root',
    database='task'
    
)
cursor=conn.cursor()

## Storing data into database

In [24]:
# SQL insert query
insert_query = """
INSERT INTO task2 (
    country, 
    election_name, 
    province, 
    district, 
    municipality, 
    total_population, 
    number_of_wards, 
    election_center, 
    number_of_male_voters, 
    number_of_female_voters,
    number_of_other_voters,
    total_eligible_voters
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
"""

# Iterate over DataFrame rows and execute insert query
for i, row in df.iterrows():
    cursor.execute(insert_query, (
        row['country'],
        row['election name'],
        row['province'],
        row['district'],
        row['municipality'],
        row['total population'],
        row['number of wards'],
        row['election center'],
        row['number of male voters'],
        row['number of female voters'],
        row['number of other voters'],
        row['total eligible voters']
    ))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()